In [1]:
import matplotlib as plt
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}

In [3]:
## Preço máximo
pmax = 550000

## Área mínima
amin = 40

## Quantidade de imóveis disponíveis
#int(soup.find('h1', class_='js-summary-title heading-regular heading-regular__bold align-left results__title').text.split(' ')[0])

In [18]:
lista = []

for pg in range(10):
    url = 'https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro+zona-sul+botafogo/?pagina='+str(pg)+'&onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.951193,-43.180784&transacao=Venda&precoMaximo='+str(pmax)+'&areaMinima='+str(amin)+'&tipo=Im%C3%B3vel%20usado'
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    containers = soup.find_all('div', class_='box--display-flex box--flex-column gutter-top-double gutter-left-double gutter-right-double gutter-bottom-double simple-card__box')
    
    for i in containers:
        ## Hash card
        hash_= hash(i)
        ## Get address OK
        address = i.find('p', class_='color-dark text-regular simple-card__address').text
        street = address.split(', ')[0]
        region = address.split(', ',1)[1]
        ## Get condominium OK
        condt = i.find(class_='card-price__item condominium text-regular')
        try:
            cond = int(str(condt).split('R$ ')[1].split('<')[0])
        except:
            cond = None
        ## Get IPTU OK
        iptut = i.find(class_='card-price__item iptu text-regular')
        try:
            iptu = int(str(iptut).split('R$ ')[1].split('<')[0])
        except:
            iptu = None
        ## Get bedrooms OK
        bdrms = i.find('li', class_='feature__item text-small js-bedrooms')
        try:
            bedrooms = int(bdrms.text.strip().split(' ')[0])
        except:
            bedrooms = None
        ## Get size OK
        info = i.find_all('li', 'feature__item text-small')
        size = int(info[0].span.text.strip().split(' ')[0])
        ## Get restrooms OK
        try:
            restrooms = int(info[2].span.text.strip().split(' ')[0]) if len(info) == 3 else int(info[1].span.text.strip())
        except:
            restrooms = None
        ## Get parking OK
        parking = int(info[1].span.text.strip().split(' ')[0]) if len(info) == 3 else 0
        ## Get price OK
        price = int(i.find('p', class_='simple-card__price js-price heading-regular heading-regular__bolder align-left').text.strip().split('R$ ')[1].replace(".",""))
        ## Get comments
        comment = i.find('span', class_='text-regular simple-card__text')
        ## Generate list
        lista.append([street, region, size, bedrooms, restrooms, parking, cond, iptu, price, hash_])

## List to dataframe
a = pd.DataFrame(lista, columns = ['Street', 'Region', 'Size', 'Bedrooms', 'Restrooms', 'Parking', 'Cond', 'IPTU', 'Price', 'Hash'])
a = a.drop_duplicates(inplace=False)
## m² Price
a['Price/m2'] = (a['Price'] / a['Size']).round(0)

## Only selected region
a = a[a['Region'] == 'Botafogo']

print('cell done')

cell done


In [54]:
## Filters
size_f = a['Size'] > 50
cond_f = a['Cond'] <= 800
rest_f = a['Restrooms'] > 1
price_f = a['Price'] <= 520000

a = a[size_f & cond_f & rest_f & price_f].sort_values(by =['Price']).reset_index(inplace=False, drop=True)

a.reset_index(inplace=False, drop=True)
a.sort_values(by =['Price'], ascending=True)#.reset_index(inplace=False, drop=True)

print('cell done')

cell done


In [55]:
#exc_hash = [729212732937898415, 824045975498986750]
#exc_i = a.index[a['Hash'] == exc_hash

#a.drop(exc_i, inplace=True)

a

,Street,Region,Size,Bedrooms,Restrooms,Parking,Cond,IPTU,Price,Hash,Price/m2
0,Rua General Polidoro,Botafogo,73,2.0,2.0,0,500.0,504.0,450000,-3831888087329225700,6164.0
1,Praia Botafogo,Botafogo,68,2.0,2.0,0,620.0,119.0,480000,-7634882749532055219,7059.0
2,Praia Botafogo,Botafogo,62,1.0,2.0,0,560.0,699.0,495000,-6383806872496560804,7984.0
3,Rua Voluntários da Pátria,Botafogo,62,1.0,2.0,0,760.0,NaN,499000,9061867906931128105,8048.0
4,Rua Real Grandeza,Botafogo,85,2.0,2.0,1,50.0,NaN,515000,417485565571932244,6059.0
5,Rua Sorocaba,Botafogo,67,2.0,2.0,0,200.0,94.0,519900,8500014804287020819,7760.0
6,Rua Sorocaba,Botafogo,67,2.0,2.0,0,200.0,95.0,519999,-6820834040909682610,7761.0
7,Rua Voluntários da Pátria,Botafogo,75,2.0,2.0,0,770.0,873.0,519999,-6826247592518102697,6933.0
8,Rua Voluntários da Pátria,Botafogo,64,2.0,2.0,0,600.0,NaN,520000,6663026840804431954,8125.0
9,Praia Botafogo,Botafogo,63,1.0,2.0,0,520.0,754.0,520000,5997440271269529696,8254.0
